In [30]:
from transformers import (
    pipeline,
    TFAutoModelForSequenceClassification,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)

In [31]:
model_uri = "cardiffnlp/twitter-roberta-base-sentiment"

# getting pre-trained model
model = TFAutoModelForSequenceClassification.from_pretrained(model_uri)
tokenizer = AutoTokenizer.from_pretrained(model_uri)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [32]:
tokenizer([
    "I've been waiting this for my whole life",
    "I hate this world"
])

{'input_ids': [[0, 100, 348, 57, 2445, 42, 13, 127, 1086, 301, 2], [0, 100, 4157, 42, 232, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [33]:
# setting up the sentiment analyzer pipline
analyzer = pipeline(task="sentiment-analysis",
                    model=model,
                    tokenizer=tokenizer)